In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyngrok
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 7.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=bd0c51f918bcc8132a548ea17f45ac731e7acad881002cb6fc34f25f9606458c
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Final project"
!ls 

/content/drive/MyDrive/Colab Notebooks/Final project
 anchors.npy	      helpers		     'Run flask API.ipynb'    video
 blazeface.pth	      model		      static		      video_up
 FaceForensic.ipynb   Pre-Proccessing.ipynb   test_1000.pkl
'Flask API.ipynb'     README.md		      train_3200.pkl
'Flask app .py'       requirements.txt	     'Training model.ipynb'


#  Import libraries

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from keras.models import load_model
import numpy as np
import cv2
import os
from flask import Flask
from werkzeug.utils import secure_filename
import os
from tensorflow import keras
from flask import Flask, request, render_template
from werkzeug.utils import secure_filename
import dlib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok


# Display Home Page 

In [ ]:
app = Flask(__name__ , template_folder='static/templates')
ngrok.set_auth_token("29AhNyFJiBxEu0s07cR2zA8VqDQ_3wmwNStUSaHgpCiP2UNkd")
run_with_ngrok(app)
@app.route('/')
def main():
  return render_template ("Home.html")

#  Redirect To Upload 

In [ ]:


@app.route('/upload_video', methods=['POST'])
def Video():
  return render_template ("Upload_Video.html")

@app.route('/upload_image', methods=['POST'])
def image():
    return render_template('Upload_image.html')
@app.route('/About-us', methods=['POST'])
def About_us():
  return render_template('About-us.html')


# Load Model 

In [ ]:
import joblib
path = '/content/drive/MyDrive/Colab Notebooks/Final Project/finalized_model.sav'
model = joblib.load(path)

In [ ]:
import numpy as np

def azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    # Calculate the indices from the image
    y, x = np.indices(image.shape[:2])

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

    r = np.hypot(x - center[0], y - center[1])

    # Get sorted radii
    ind = np.argsort(r.flat)
    r_sorted = r.flat[ind]
    i_sorted = image.flat[ind]

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]

    radial_prof = tbin / nr

    return radial_prof

In [ ]:
import sys
import os
import torch
import cv2
import time
from torch import nn
import matplotlib.pyplot as plt
import matplotlib
from torchvision.transforms import Normalize
gpu = torch.device('cuda:0'if torch.cuda.is_available() else 'cpu')
gpu

root = '/content/drive/MyDrive'

frames_per_video = 100
input_size = 224
test_val_frac = 0.3

sys.path.insert(0, os.path.join(root, 'Colab Notebooks', 'Final project', 'blazeface-pytorch'))
sys.path.insert(0, os.path.join(root, 'Colab Notebooks', 'Final project'))

from blazeface import BlazeFace
facedet = BlazeFace().to(gpu)
facedet.load_weights(os.path.join(root, 'Colab Notebooks', 'Final project', "blazeface.pth"))
facedet.load_anchors(os.path.join(root, 'Colab Notebooks', 'Final project', "anchors.npy"))

_ = facedet.train(False)
from helpers.read_video_1 import VideoReader
from helpers.face_extract_1 import FaceExtractor

video_reader = VideoReader(verbose=True)
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [ ]:
def add_margin_to_detections(detections, frame_size, margin=0.2):
        offset = torch.round(margin * (detections[:, 2] - detections[:, 0]))
        detections = detections.clone()
        detections[:, 0] = torch.clamp(detections[:, 0] - offset*2, min=0)            # ymin
        detections[:, 1] = torch.clamp(detections[:, 1] - offset, min=0)              # xmin
        detections[:, 2] = torch.clamp(detections[:, 2] + offset, max=frame_size[1])  # ymax
        detections[:, 3] = torch.clamp(detections[:, 3] + offset, max=frame_size[0])  # xmax
        return detections
    
def crop_faces(img, detections ):
    """Copies the face region(s) from the given frame into a set
    of new NumPy arrays.

    Arguments:
        frame: a NumPy array of shape (H, W, 3)
        detections: a PyTorch tensor of shape (num_detections, 17)

    Returns a list of NumPy arrays, one for each face crop. If there
    are no faces detected for this frame, returns an empty list.
    """
    faces = []
    for i in range(len(detections)):
        ymin, xmin, ymax, xmax = detections[i, :4].cpu().numpy()
        ymin = ymin * img.shape[0]
        xmin = xmin * img.shape[1]
        ymax = ymax * img.shape[0]
        xmax = xmax * img.shape[1]
        face = img[ymin.astype(np.int):ymax.astype(np.int), xmin.astype(np.int):xmax.astype(np.int), :]
        faces.append(face)
    return faces

def keep_only_best_face(crops):
        for i in range(len(crops)):
            frame_data = crops[i]
            if len(frame_data["faces"]) > 0:
                frame_data["faces"] = frame_data["faces"][:1]
                frame_data["scores"] = frame_data["scores"][:1]


def detec_and_crop_best_face(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  img = cv2.resize(img, facedet.input_size, interpolation = cv2.INTER_AREA)

  target_size = facedet.input_size
  frame_size = (img.shape[1], img.shape[0])

  detections = facedet.predict_on_image(img)
  faces = add_margin_to_detections(detections, frame_size, 0.5)
  scores = detections[:, 16].cpu().numpy()

  faces = crop_faces(img, faces)

  return faces[np.where(scores == np.amax(scores))[0][0]], scores[np.where(scores == np.amax(scores))[0][0]]

# Function Prediction 

In [ ]:
N = 500
from google.colab.patches import cv2_imshow

from scipy.interpolate import griddata
def prediction_img (filepath, rate=0.1):
  img, score = detec_and_crop_best_face(filepath)

  print(score)
  if(score < rate):
    return 'Không có mặt người trong ảnh'
  # we crop the center
  h = int(img.shape[0]/3)
  w = int(img.shape[1]/3)
  img = img[h:-h,w:-w]

  f = np.fft.fft2(img)
  fshift = np.fft.fftshift(f)

  magnitude_spectrum = 20*np.log(np.abs(fshift))
  psd1D = azimuthalAverage(magnitude_spectrum)

  # Calculate the azimuthally averaged 1D power spectrum
  points = np.linspace(0,N,num=psd1D.size) # coordinates of a
  xi = np.linspace(0,N,num=N) # coordinates for interpolation

  interpolated = griddata(points,psd1D,xi,method='cubic')
  interpolated /= interpolated[0]
  pre = model.predict([interpolated])
  if(pre == 1):
    return 'Real'
  else:
    return 'Fake'

def prediction (filepath, rate =0.95):
  check = 0
  faces = face_extractor.process_video(filepath)
  face_extractor.keep_only_best_face(faces)
  if len(faces)>0:
    num=0
    for frame_data in faces:
      scores = frame_data['scores']
      for ind, img in enumerate(frame_data['faces']):
        if(scores[ind] > rate):
          # cv2_imshow(img)
          img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
          # cv2_imshow(img)
          h = int(img.shape[0]/3)
          w = int(img.shape[1]/3)
          img = img[h:-h,w:-w]

          f = np.fft.fft2(img)
          fshift = np.fft.fftshift(f)

          magnitude_spectrum = 20*np.log(np.abs(fshift))
          psd1D = azimuthalAverage(magnitude_spectrum)

          # Calculate the azimuthally averaged 1D power spectrum
          points = np.linspace(0,N,num=psd1D.size) # coordinates of a
          xi = np.linspace(0,N,num=N) # coordinates for interpolation

          interpolated = griddata(points,psd1D,xi,method='cubic')
          interpolated /= interpolated[0]
          if(int(model.predict([interpolated]))):
            check -=1
          else: check +=1
  if(check <0):
    return('Fake')
  else:
    return("Real")

# Upload File

In [ ]:
UPLOAD_FOLDER = 'static'
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

#  Video Prediction & Display Result 

In [ ]:
@app.route('/predict_video', methods=['POST'])
def upload_video():
	file = request.files['file']
	filename = secure_filename(file.filename)
	file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
	filepath = "static/"+filename
	preds = prediction(filepath)
	return render_template("Display_Video.html",prediction =preds ,video_path = filename)

# Image Prediction & Display Result 

In [ ]:
@app.route('/Predict_image', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        f = request.files['file']
        file_path = os.path.join (app.config['UPLOAD_FOLDER'], secure_filename(f.filename))
        f.save(file_path)
        
        preds = prediction_img(file_path)
    return render_template("Display_image.html",prediction = preds, img_path= f.filename )


# **Run Flask Application**#  

In [ ]:
if __name__ =='__main__': 
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8df6-35-199-185-69.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:22:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:22:59] "GET /static/css/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:22:59] "GET /static/css/normalize.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:23:00] "GET /static/background/cover.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:23:00] "GET /static/background/cover.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:23:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:23:01] "GET /static/background/cover.mp4 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:23:12] "POST /upload_image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:23:12] "GET /static/js/jquery-1.12.4.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:23:12] "GET /static/js/main.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Nov/2022 07:23:13] "GET /static/c